# 6.2 Logistic Regression Pipeline

##### Description

Pipelines to train the models are implemented in this notebook. The preprocessing steps of scaling and encoding are embedded in the pipeline. The exploratory models are trained to determine which has the highest accuracy score.

##### Notebook Steps

1. Connect Spark
1. Input data
1. Basic data review
1. Visualize relationships

## 1. Connect Spark

In [ ]:
%load_ext sparkmagic.magics

In [ ]:
%manage_spark

In [ ]:
%%spark
spark.sparkContext.setCheckpointDir('./checkpoints')

## 2. Load Data
There are two seperate datasets we will be working with while modeling. They are train.csv and validate.csv. Train is used to train the model, while validate is used to perform evaluation on unseen data. Only train is needed at this time.

In [ ]:
%%spark
train = spark.read.csv("s3://jolfr-capstone3/training/train.csv", header=True, inferSchema=True)

In [ ]:
%%spark

train, drop = train.randomSplit(weights = [0.25, 0.75], seed = 42)

## 4. Define Pipeline Steps

 1. Feature Hasher
 1. Standard Scaler
 1. Model

Collects column names for later use in pipeline.

In [ ]:
%%spark
cols = train.drop("label").columns

##### Define Feature Hasher

In [ ]:
%%spark
from pyspark.ml.feature import FeatureHasher
hasher = FeatureHasher(inputCols=cols, outputCol="hash")

##### Define Scaler

In [ ]:
%%spark
from pyspark.ml.feature import StandardScaler
scaler = StandardScaler(inputCol=hasher.getOutputCol(), outputCol="features")

## 5. Define Model and Parameter Grid

In [ ]:
%%spark
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.tuning import ParamGridBuilder

lr = LogisticRegression(maxIter=10)

params = ParamGridBuilder().addGrid(lr.regParam, [0.1, 0.01]).addGrid(lr.elasticNetParam, [0.4, 0.8]).build()

## 6. Define Pipeline

In [ ]:
%%spark
from pyspark.ml import Pipeline
pipe = Pipeline(stages=[hasher, scaler, lr])

## 6. Define Cross Validator

In [ ]:
%%spark
from pyspark.ml.tuning import CrossValidator
from pyspark.ml.evaluation import BinaryClassificationEvaluator
crossval = CrossValidator(estimator=pipe, estimatorParamMaps=params, evaluator=BinaryClassificationEvaluator(), numFolds=5, parallelism=5, seed=42)

## 7. Fit Model

In [ ]:
%%spark
from pyspark.sql.types import FloatType
from pyspark.sql.functions import col
train = train.withColumn("label", col("label").cast(FloatType()))

In [ ]:
%%spark
cvModel = crossval.fit(train)

## 8. Save Best Params

In [ ]:
%%spark
best_params = cvModel.bestModel.stages[-1].extractParamMap()

params = {}

for k, v in best_params.items():
    params[k.name] = v

params

In [ ]:
%%spark
import boto3
import pickle

s3 = boto3.client('s3')

serialized = pickle.dumps(params)

s3.put_object(Bucket='jolfr-capstone3',Key='LogisticParams', Body=serialized)